In [10]:
import numpy as np 
import pandas as pd 

import os
from pathlib import Path

import string
import nltk
from nltk.corpus import stopwords

import scipy.io
import scipy.linalg
from scipy.sparse import csr_matrix, vstack, lil_matrix 
from sklearn.base import TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB

import plotly.express as px
import plotly.figure_factory as ff
from yellowbrick.text import TSNEVisualizer

In [11]:
DATASET_PATH = './measuring_hate_speech.csv'

# Data Cleaning

In [12]:
from cleaning.clean_dataset import CleanTextDatasetOperator

clean_dataset_operator = CleanTextDatasetOperator(
    file_path=DATASET_PATH
)

help(CleanTextDatasetOperator)

Help on class CleanTextDatasetOperator in module cleaning.clean_dataset:

class CleanTextDatasetOperator(builtins.object)
 |  CleanTextDatasetOperator(file_path='', *args, **kwargs)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, file_path='', *args, **kwargs)
 |      This class wraps all data cleaning logic under one shade.
 |      
 |      Args:   
 |          file_path (str) : Path to the text dataset file in csv format.
 |      Returns:
 |          cleaned_df (pandas.DataFrame) : processed cleaned data frame.
 |      
 |      Added By : Abbas Ismail
 |  
 |  expand_contractions(self, df: pandas.core.frame.DataFrame)
 |      Method for expanding compression for text column
 |      
 |      Added By : Simranjeet and Navneet kaur
 |  
 |  load_csv_to_dataset(self)
 |      This method loads CSV dataset to pandas Dataframe
 |      Returns:
 |          df : pandas.DataFrame - dataset dataframe
 |      
 |      Added By : Abbas Ismail
 |  
 |  lowercase_text(self, df: pandas.core.f

In [13]:
#Loading the dataset
dataset_df = clean_dataset_operator.load_csv_to_dataset()
dataset_df.head(10)

,hatespeech,text,sentiment,respect,insult,humiliate,dehumanize,violence,genocide,attack_defend
0,0.0,Yes indeed. She sort of reminds me of the elde...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,The trans women reading this tweet right now i...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,2.0,Question: These 4 broads who criticize America...,4.0,4.0,4.0,4.0,4.0,0.0,0.0,4.0
3,0.0,It is about time for all illegals to go back t...,2.0,3.0,2.0,1.0,0.0,0.0,0.0,3.0
4,2.0,For starters bend over the one in pink and kic...,4.0,4.0,4.0,4.0,4.0,4.0,1.0,3.0
5,0.0,Sounds like the kinda wholsesome life I'd die ...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
6,2.0,Who the fuck is this insignificant simple mind...,4.0,4.0,4.0,4.0,3.0,0.0,0.0,3.0
7,2.0,Fuck off you insufferable retarded faggot.,4.0,4.0,4.0,4.0,0.0,0.0,0.0,3.0
8,0.0,"OMG! The EGO's of these young, young, inexperi...",4.0,4.0,4.0,3.0,3.0,0.0,1.0,1.0
9,2.0,"Worthless whore, these tits with look nice wit...",4.0,4.0,4.0,4.0,4.0,4.0,0.0,4.0


In [14]:
#changing the text to lowercase 
dataset_df = clean_dataset_operator.lowercase_text(df=dataset_df)
dataset_df.head(10)

,hatespeech,text,sentiment,respect,insult,humiliate,dehumanize,violence,genocide,attack_defend
0,0.0,yes indeed. she sort of reminds me of the elde...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,the trans women reading this tweet right now i...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,2.0,question: these 4 broads who criticize america...,4.0,4.0,4.0,4.0,4.0,0.0,0.0,4.0
3,0.0,it is about time for all illegals to go back t...,2.0,3.0,2.0,1.0,0.0,0.0,0.0,3.0
4,2.0,for starters bend over the one in pink and kic...,4.0,4.0,4.0,4.0,4.0,4.0,1.0,3.0
5,0.0,sounds like the kinda wholsesome life i'd die ...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
6,2.0,who the fuck is this insignificant simple mind...,4.0,4.0,4.0,4.0,3.0,0.0,0.0,3.0
7,2.0,fuck off you insufferable retarded faggot.,4.0,4.0,4.0,4.0,0.0,0.0,0.0,3.0
8,0.0,"omg! the ego's of these young, young, inexperi...",4.0,4.0,4.0,3.0,3.0,0.0,1.0,1.0
9,2.0,"worthless whore, these tits with look nice wit...",4.0,4.0,4.0,4.0,4.0,4.0,0.0,4.0


In [15]:
#Remove numbers 
dataset_df['text'] = clean_dataset_operator.remove_number(df=dataset_df[['text']])
dataset_df.dropna(inplace=True)
dataset_df.head(10)

/Users/aismail/Desktop/final/NLP-Group5/cleaning/clean_dataset.py:76: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[columns_with_numbers] = df[columns_with_numbers].applymap(lambda x: None if contains_numbers(x) else x)


,hatespeech,text,sentiment,respect,insult,humiliate,dehumanize,violence,genocide,attack_defend
0,0.0,yes indeed. she sort of reminds me of the elde...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,the trans women reading this tweet right now i...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,0.0,it is about time for all illegals to go back t...,2.0,3.0,2.0,1.0,0.0,0.0,0.0,3.0
4,2.0,for starters bend over the one in pink and kic...,4.0,4.0,4.0,4.0,4.0,4.0,1.0,3.0
5,0.0,sounds like the kinda wholsesome life i'd die ...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
6,2.0,who the fuck is this insignificant simple mind...,4.0,4.0,4.0,4.0,3.0,0.0,0.0,3.0
7,2.0,fuck off you insufferable retarded faggot.,4.0,4.0,4.0,4.0,0.0,0.0,0.0,3.0
8,0.0,"omg! the ego's of these young, young, inexperi...",4.0,4.0,4.0,3.0,3.0,0.0,1.0,1.0
9,2.0,"worthless whore, these tits with look nice wit...",4.0,4.0,4.0,4.0,4.0,4.0,0.0,4.0
11,0.0,"instagram refugees lmao, let's build a wall in...",3.0,3.0,3.0,3.0,2.0,1.0,1.0,2.0


In [16]:
#Remove numbers 
nltk.download('stopwords')
dataset_df['text'] = clean_dataset_operator.remove_stopwords(df=dataset_df[['text']])
dataset_df.dropna(inplace=True)
dataset_df.head(10)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aismail/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,hatespeech,text,sentiment,respect,insult,humiliate,dehumanize,violence,genocide,attack_defend
0,0.0,yes indeed . sort reminds elder lady played pa...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,trans women reading tweet right beautiful,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,0.0,time illegals go back country origin keep free...,2.0,3.0,2.0,1.0,0.0,0.0,0.0,3.0
4,2.0,starters bend one pink kick ass pussy get tast...,4.0,4.0,4.0,4.0,4.0,4.0,1.0,3.0
5,0.0,sounds like kinda wholsesome life 'd die ❤️ ne...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
6,2.0,fuck insignificant simple minded redneck ? get...,4.0,4.0,4.0,4.0,3.0,0.0,0.0,3.0
7,2.0,fuck insufferable retarded faggot .,4.0,4.0,4.0,4.0,0.0,0.0,0.0,3.0
8,0.0,"omg ! ego 's young , young , inexperienced wom...",4.0,4.0,4.0,3.0,3.0,0.0,1.0,1.0
9,2.0,"worthless whore , tits look nice bite marks cum",4.0,4.0,4.0,4.0,4.0,4.0,0.0,4.0
11,0.0,"instagram refugees lmao , let 's build wall in...",3.0,3.0,3.0,3.0,2.0,1.0,1.0,2.0


In [17]:
nltk.download('punkt')
def get_summary(df):   

    content = df["text"].values        
    word_tok = [word.lower() for item in content for word in nltk.word_tokenize(item)]    
    st_words = set(word_tok)   
    
    fact = {
        "TotalCount": len(content),
        "TotalWords": len(word_tok),        
        "TotalUniqueWords": len(st_words),
        "MeanWordsPerTweet": len(word_tok) / len(content),
    }

    return fact, df.describe()

[nltk_data] Downloading package punkt to /Users/aismail/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
f, s = get_summary(dataset_df)

In [19]:
f

{'TotalCount': 117920,
 'TotalWords': 2124197,
 'TotalUniqueWords': 40115,
 'MeanWordsPerTweet': 18.013882293080055}

In [20]:
s

,hatespeech,sentiment,respect,insult,humiliate,dehumanize,violence,genocide,attack_defend
count,117920.000000,117920.000000,117920.000000,117920.000000,117920.000000,117920.000000,117920.000000,117920.000000,117920.000000
mean,0.757378,2.956852,2.839086,2.578333,2.296158,1.863526,1.045310,0.663891,2.634218
std,0.935394,1.242679,1.318359,1.394268,1.375813,1.410096,1.341694,1.165313,1.119751
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,2.000000,2.000000,1.000000,1.000000,0.000000,0.000000,2.000000
50%,0.000000,3.000000,3.000000,3.000000,3.000000,2.000000,0.000000,0.000000,3.000000
75%,2.000000,4.000000,4.000000,4.000000,3.000000,3.000000,2.000000,1.000000,4.000000
max,2.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000


In [21]:
# Remove stop words, special chars 
# stem the word tokens
# re.sub(r'^https?:\/\/.*[\r\n]*', '', text)
def clean_tweet(sent):
    stemmer = nltk.PorterStemmer()        
    tknzr = nltk.RegexpTokenizer(r'[a-zA-Z0-9]+')

    exclp = list(string.punctuation)     
    exclc = [
        "'re", "n't", "'m", "'s", "n't", "'s", 
        "``", "''", "'ve", "'m", "'ll", "'ve", 
        "...", "http", "https"]    
    sw = set(stopwords.words("english") + exclp + exclc)    

    tokens = tknzr.tokenize(sent.lower())
    words = [stemmer.stem(token) for token in tokens if not token in sw]
    return " ".join(words)

dataset_df['text'] = dataset_df['text'].apply(lambda x : clean_tweet(x))

In [22]:
dataset_df

,hatespeech,text,sentiment,respect,insult,humiliate,dehumanize,violence,genocide,attack_defend
0,0.0,ye inde sort remind elder ladi play part movi ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,tran women read tweet right beauti,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,0.0,time illeg go back countri origin keep freeway...,2.0,3.0,2.0,1.0,0.0,0.0,0.0,3.0
4,2.0,starter bend one pink kick ass pussi get tast ...,4.0,4.0,4.0,4.0,4.0,4.0,1.0,3.0
5,0.0,sound like kinda wholsesom life die never met ...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...
135544,0.0,beyond race religion polit interest basic huma...,1.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0
135547,1.0,op realli hope commit suicid one day die ass c...,3.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0
135548,1.0,dssupliftsthethirdgend emancipationofeunuch eu...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
135549,0.0,umar khalid sahela rashid kanhiyan kumar hate ...,3.0,3.0,2.0,2.0,2.0,2.0,0.0,2.0


In [23]:
def show_wfreq_plot(df, label, labelDescr = ""):   
    xdf = df[df["hatespeech"] == label]
    content = xdf["text"].values        
    word_tok = [word.lower() for item in content for word in nltk.word_tokenize(item)]    
    st_words = set(word_tok)   
    freq_dist = nltk.FreqDist(word_tok)    
    ls_freq = [(word, frequency) for word, frequency in freq_dist.most_common(20)]
    twdf = pd.DataFrame(ls_freq, columns=["Word", "Frequency"])
    tfig = px.bar(twdf, x="Word", y="Frequency", title="Top 20 most frequent words - " + labelDescr)
    tfig.show()

In [24]:
show_wfreq_plot(dataset_df, 2, "hatespeech")

In [25]:
show_wfreq_plot(dataset_df, 1, "hatespeech")

In [39]:
from sklearn.model_selection import train_test_split

def get_train_test_data_and_feature_selection(x_col='text', y_col='hatespeech'):

    #split the data 

    seed = 51
    test_size = 0.2 #20% of the data in the 
    X = dataset_df[x_col]
    y = dataset_df[y_col]

    # x_test = X[:20000]
    # y_test = y[:20000]

    # x_train= X[20000:]
    # y_train = y[20000:]

    x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=seed)
    print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)



    #instantiate the vectorizer 
    vectorizer = TfidfVectorizer()


    #fit on the training data
    x_train = vectorizer.fit_transform(x_train)
    #transform the test data
    x_test = vectorizer.transform(x_test)

    filename = 'model_vectorizer.sav'
    pickle.dump(vectorizer, open(filename, 'wb'))


    return x_train, x_test, y_train, y_test

In [27]:
x_train, x_test, y_train, y_test = get_train_test_data_and_feature_selection()

(106128,) (11792,) (106128,) (11792,)


In [28]:
from sklearn.ensemble import RandomForestClassifier

def run_logreg(df, x_train, x_test, y_train, y_test):     
    
    clf = LogisticRegression(
        penalty='l2', random_state=0, 
        solver='liblinear', max_iter=1000,
    )      
    clf.fit(x_train, y_train)  
    predicted = clf.predict(x_test)           
    return clf, predicted, y_test

def run_linsvm(df, x_train, x_test, y_train, y_test):  
    
    clf = LinearSVC(
        C=0.1,
        max_iter=1000
    )       
    clf.fit(x_train, y_train)  
    predicted = clf.predict(x_test)           
    return clf, predicted, y_test


def run_forest(df, x_train, x_test, y_train, y_test): 

    clf = RandomForestClassifier()       
    clf.fit(x_train, y_train)  
    predicted = clf.predict(x_test)           
    return clf, predicted, y_test


In [29]:
def show_results(predicted, y_test, labels=[0,1,2],  label_descr=["Hate", "Offensive", "Neither"]):
    clsr = classification_report(y_test, predicted, target_names=labels, output_dict=True)
    cm = confusion_matrix(y_test, predicted, labels=labels)   
    
    cr_df = pd.DataFrame(clsr).transpose()    
    print(cr_df)
    
    fig = ff.create_annotated_heatmap(cm, x=label_descr, y=label_descr)
    fig.update_layout(title_text='Confusion Matrix')
    fig.show()

In [30]:
clf, pred, y_test = run_logreg(dataset_df, x_train, x_test, y_train, y_test)
show_results(pred, y_test)

              precision    recall  f1-score       support
0              0.791381  0.899064  0.841793   6945.000000
1              0.000000  0.000000  0.000000    752.000000
2              0.761661  0.725763  0.743279   4095.000000
accuracy       0.781547  0.781547  0.781547      0.781547
macro avg      0.517681  0.541609  0.528357  11792.000000
weighted avg   0.730592  0.781547  0.753899  11792.000000


/Users/aismail/Desktop/air/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/aismail/Desktop/air/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/aismail/Desktop/air/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [31]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(clf, open(filename, 'wb'))

In [32]:
filename = 'model_vectorizer.sav'
pickle.dump(vectorizer, open(filename, 'wb'))

In [33]:
def reduce_categories(number):
    return number // 2

In [34]:
dataset_df.columns

Index(['hatespeech', 'text', 'sentiment', 'respect', 'insult', 'humiliate',
       'dehumanize', 'violence', 'genocide', 'attack_defend'],
      dtype='object')

In [35]:
for col in ['sentiment', 'respect', 'insult', 'humiliate','dehumanize', 'violence', 'genocide', 'attack_defend']:  
    dataset_df[col] = dataset_df[col].apply(reduce_categories)

In [36]:
dataset_df

,hatespeech,text,sentiment,respect,insult,humiliate,dehumanize,violence,genocide,attack_defend
0,0.0,ye inde sort remind elder ladi play part movi ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,tran women read tweet right beauti,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,time illeg go back countri origin keep freeway...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
4,2.0,starter bend one pink kick ass pussi get tast ...,2.0,2.0,2.0,2.0,2.0,2.0,0.0,1.0
5,0.0,sound like kinda wholsesom life die never met ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
135544,0.0,beyond race religion polit interest basic huma...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
135547,1.0,op realli hope commit suicid one day die ass c...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
135548,1.0,dssupliftsthethirdgend emancipationofeunuch eu...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
135549,0.0,umar khalid sahela rashid kanhiyan kumar hate ...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0


In [37]:
x_train, x_test, y_train, y_test = get_train_test_data_and_feature_selection(x_col='text', y_col='sentiment')

(106128,) (11792,) (106128,) (11792,)


In [38]:
clf_sentiment, pred, y_test = run_logreg(dataset_df, x_train, x_test, y_train, y_test)
show_results(pred, y_test)

              precision    recall  f1-score       support
0              0.805124  0.616895  0.698552   1681.000000
1              0.654500  0.723643  0.687337   4733.000000
2              0.788845  0.773150  0.780918   5378.000000
accuracy       0.731004  0.731004  0.731004      0.731004
macro avg      0.749490  0.704562  0.722269  11792.000000
weighted avg   0.737243  0.731004  0.731616  11792.000000


In [40]:
def create_all_models(y_col = []):
   model_dict = {}

   model_output = {}

   for col in y_col:
        x_tarin_test_tuple = get_train_test_data_and_feature_selection(x_col='text', y_col=col)
        model_dict[col] = run_logreg(dataset_df, x_tarin_test_tuple[0], x_tarin_test_tuple[1], x_tarin_test_tuple[2], x_tarin_test_tuple[3])

        show_results(model_dict[col][1], model_dict[col][2])


        model_output[col] = model_dict[col][0].predict(x_tarin_test_tuple[0])

   for col in y_col:
      print(model_output[col].shape)

Creating SubModels Using Other Independent features

In [41]:
create_all_models(y_col=['sentiment', 'respect', 'insult', 'humiliate','dehumanize', 'violence', 'genocide', 'attack_defend'])

(106128,) (11792,) (106128,) (11792,)
              precision    recall  f1-score       support
0              0.805124  0.616895  0.698552   1681.000000
1              0.654500  0.723643  0.687337   4733.000000
2              0.788845  0.773150  0.780918   5378.000000
accuracy       0.731004  0.731004  0.731004      0.731004
macro avg      0.749490  0.704562  0.722269  11792.000000
weighted avg   0.737243  0.731004  0.731616  11792.000000


(106128,) (11792,) (106128,) (11792,)
              precision    recall  f1-score       support
0              0.774817  0.596410  0.674008   1950.000000
1              0.636703  0.730736  0.680486   4672.000000
2              0.796510  0.759381  0.777503   5170.000000
accuracy       0.721082  0.721082  0.721082      0.721082
macro avg      0.736010  0.695509  0.710666  11792.000000
weighted avg   0.729607  0.721082  0.721950  11792.000000


(106128,) (11792,) (106128,) (11792,)
              precision    recall  f1-score       support
0              0.754519  0.638336  0.691582   2812.000000
1              0.604737  0.712730  0.654308   5051.000000
2              0.719075  0.633240  0.673433   3929.000000
accuracy       0.668504  0.668504  0.668504      0.668504
macro avg      0.692777  0.661435  0.673108  11792.000000
weighted avg   0.678552  0.668504  0.669569  11792.000000


(106128,) (11792,) (106128,) (11792,)
              precision    recall  f1-score       support
0              0.738169  0.600618  0.662328   3558.000000
1              0.594786  0.728618  0.654935   5542.000000
2              0.595825  0.466568  0.523333   2692.000000
accuracy       0.630173  0.630173  0.630173      0.630173
macro avg      0.642927  0.598601  0.613532  11792.000000
weighted avg   0.638286  0.630173  0.627122  11792.000000


(106128,) (11792,) (106128,) (11792,)
              precision    recall  f1-score       support
0              0.661711  0.700038  0.680336   5214.000000
1              0.506301  0.537781  0.521566   4632.000000
2              0.565634  0.394142  0.464567   1946.000000
accuracy       0.585821  0.585821  0.585821      0.585821
macro avg      0.577882  0.543987  0.555490  11792.000000
weighted avg   0.584809  0.585821  0.582362  11792.000000


(106128,) (11792,) (106128,) (11792,)
              precision    recall  f1-score       support
0              0.836802  0.969286  0.898185   8628.000000
1              0.517661  0.208845  0.297619   2035.000000
2              0.715455  0.619132  0.663818   1129.000000
accuracy       0.804528  0.804528  0.804528      0.804528
macro avg      0.689973  0.599088  0.619874  11792.000000
weighted avg   0.770108  0.804528  0.772104  11792.000000


(106128,) (11792,) (106128,) (11792,)
              precision    recall  f1-score       support
0              0.908594  0.986910  0.946134   9931.000000
1              0.404372  0.068015  0.116444   1088.000000
2              0.673966  0.716688  0.694671    773.000000
accuracy       0.884413  0.884413  0.884413      0.884413
macro avg      0.662310  0.590538  0.585749  11792.000000
weighted avg   0.846691  0.884413  0.853098  11792.000000


(106128,) (11792,) (106128,) (11792,)
              precision    recall  f1-score       support
0              0.660138  0.342703  0.451181   1672.000000
1              0.725440  0.873483  0.792608   7169.000000
2              0.723386  0.561843  0.632462   2951.000000
accuracy       0.720234  0.720234  0.720234      0.720234
macro avg      0.702988  0.592677  0.625417  11792.000000
weighted avg   0.715667  0.720234  0.704120  11792.000000


(106128,)
(106128,)
(106128,)
(106128,)
(106128,)
(106128,)
(106128,)
(106128,)
